In [1]:
# Importações
import random
import math
import numpy as np
import pandas as pd
import requests
import time
import warnings
from deap import base, creator, tools, algorithms

# Suprimir todas as mensagens de aviso
warnings.filterwarnings("ignore")

# GA

No entanto, o algoritmo que é frequentemente usado em otimização de portfólios é o Algoritmo Genético (GA).
O GA é amplamente adotado na área de finanças quantitativas devido à sua capacidade de lidar com problemas complexos e não lineares,
como a seleção de ativos em um portfólio. Ele tem a flexibilidade de ser ajustado para atender às necessidades específicas de um investidor
em termos de restrições, preferências de risco e retorno.

- Tive problemas com o Acesso à API, porque eu tava pegando uma API Premium ('TIME_SERIES_DAILY_ADJUSTED')
- Usei Thundercliente para testar API
-

In [7]:
# Função para calcular o retorno e o risco de um portfólio
def evaluate_portfolio(individual, data):
    weights = np.array(individual)

    # Verifique se há dados disponíveis antes de calcular os índices de máximo
    if data.empty or data.isnull().values.any():
        print("Dados ausentes ou nulos. Retornando -inf, inf.")
        return -np.inf, np.inf

    # Verifique se há pelo menos dois dias de dados para calcular o retorno
    if len(data) < 2:
        print("Menos de dois dias de dados disponíveis. Retornando -inf, inf.")
        return -np.inf, np.inf

    try:
        returns = np.sum(data.pct_change().mean() * weights) * 252
        volatility = np.sqrt(np.dot(weights.T, np.dot(data.pct_change().cov() * 252, weights)))
        print("Cálculos de retorno e volatilidade bem-sucedidos.")
        return returns, volatility
    except Exception as e:
        print(f"Erro durante o cálculo: {e}")
        return -np.inf, np.inf

# Função para obter os dados de preços de fechamento de ativos da Alpha Vantage
def get_alpha_vantage(symbols, api_key):
  data = {}

  for symbol in symbols:
    base_url = 'https://www.alphavantage.co/query'
    params = {
      'function': 'TIME_SERIES_DAILY',
      'symbol': symbol,
      'apikey': api_key
    }

    try:
      response = requests.get(base_url, params=params)
      raw_data = response.json()
      time_series = raw_data.get('Time Series (Daily)', {})

      # Armazena os dados em um dicionário
      data[symbol] = {date: float(info['4. close']) for date, info in time_series.items()}
    except Exception as e:
      print(f"Erro ao obter dados para {symbol}: {e}")

  return data

def genetic_algorithm(n_assets, data):
  # Defina os parâmetros do algoritmo genético
  creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))  # Maximizar retorno e minimizar risco
  creator.create("Individual", list, fitness=creator.FitnessMulti)

  toolbox = base.Toolbox()
  toolbox.register("attr_float", np.random.uniform, 0, 1)
  toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=n_assets)
  toolbox.register("population", tools.initRepeat, list, toolbox.individual)
  toolbox.register("evaluate", evaluate_portfolio, data=data)
  toolbox.register("mate", tools.cxBlend, alpha=0.5)
  toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.2)
  toolbox.register("select", tools.selNSGA2)

  # Crie a população inicial
  population = toolbox.population(n=100)

  # Execute o algoritmo genético
  n_generations = 50

  for gen in range(n_generations):
      offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.2)
      fits = toolbox.map(toolbox.evaluate, offspring)

      for fit, ind in zip(fits, offspring):
          ind.fitness.values = fit

      population = toolbox.select(offspring, len(population))

  # Recupere a solução ótima (portfólio)
  return tools.selBest(population, 1)[0]

def main():
  # Símbolos dos ativos que você deseja incluir no portfólio
  assets = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA"]
  n_assets = len(assets)

  # Substitua 'YOUR_API_KEY' pela sua chave de API Alpha Vantage
  api_key = '4KR9RHTR37YQ1XY5'

  # Obtém os dados de preços de fechamento dos ativos
  asset_data = get_alpha_vantage(assets, api_key)

  # Cria um DataFrame a partir dos dados obtidos
  data = pd.DataFrame(asset_data)

  start_time = time.time()
  best_portfolio = genetic_algorithm(n_assets, data)
  end_time = time.time()

  print("Melhor alocação de ativos:")
  print(assets)
  print(best_portfolio)
  best_return, best_risk = evaluate_portfolio(best_portfolio, data)
  print(f"Retorno anual: {best_return * 100:.2f}%")
  print(f"Risco anual: {best_risk * 100:.2f}%")
  print(f"Tempo de execução: {end_time - start_time:.2f} segundos")

if __name__ == "__main__":
  main()

Melhor alocação de ativos:
['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
[-1.4384749740684895, 0.09942430016108933, -0.007618628695249652, -0.5399171618032805, -2.202065815453543]
Retorno anual: -inf%
Risco anual: inf%
Tempo de execução: 0.06 segundos


# SA (Simulated Annealing)

## Desafios Enfrentados

> Um dos desafios iniciais foi obter dados históricos de preços de fechamento de ativos financeiros. A fonte de dados utilizada foi a Alpha Vantage, e o desafio estava em garantir que os dados estivessem disponíveis e fossem precisos.

> A implementação do algoritmo Simulated Annealing do zero exigiu um entendimento sólido da teoria por trás do algoritmo e a tradução desse conhecimento em código funcional. Isso incluiu a seleção apropriada da função de resfriamento, a criação de uma estrutura para perturbar as soluções e a definição correta dos critérios de aceitação de soluções piores.

> A definição de parâmetros, como a temperatura inicial, a taxa de resfriamento e o número de iterações, foi um desafio. Encontrar um equilíbrio entre explorar o espaço de soluções e convergir para uma solução ótima exigiu iterações e ajustes.


## Decisões Tomadas

> A linguagem de programação escolhida foi Python devido à sua eficácia no tratamento de dados e à disponibilidade de bibliotecas relevantes, como NumPy e pandas.

> Optou-se pelo Simulated Annealing como método de otimização, pois ele é adequado para problemas complexos de otimização, como a alocação de ativos em um portfólio.

> Decidiu-se por uma abordagem simples de novas soluções das soluções atuais, trocando aleatoriamente o peso de um ativo em cada iteração. Isso permitiu explorar o espaço de soluções de forma eficaz.

## Resultados Obtidos

> Os resultados finais do projeto incluem a alocação ótima de ativos em um portfólio que maximiza o retorno anual e minimiza o risco anual. Os resultados podem variar dependendo dos parâmetros de entrada e da aleatoriedade introduzida pelo processo do Simulated Annealing.

In [3]:
# Função para calcular o retorno e o risco de um portfólio
def evaluate_portfolio(individual, data):
  weights = np.array(individual)
  returns = np.sum(data.pct_change().mean() * weights) * 252
  volatility = np.sqrt(np.dot(weights.T, np.dot(data.pct_change().cov() * 252, weights)))
  return returns, volatility

# Função de resfriamento exponencial para o Simulated Annealing
def exponential_decay(temp, cooling_rate):
  return temp * cooling_rate

# Função para obter os dados de preços de fechamento de ativos da Alpha Vantage
def get_alpha_vantage(symbols, api_key):
  data = {}

  for symbol in symbols:
    base_url = 'https://www.alphavantage.co/query'
    params = {
      'function': 'TIME_SERIES_DAILY',
      'symbol': symbol,
      'apikey': api_key
    }

    try:
      response = requests.get(base_url, params=params)
      raw_data = response.json()
      time_series = raw_data.get('Time Series (Daily)', {})

      # Armazena os dados em um dicionário
      data[symbol] = {date: float(info['4. close']) for date, info in time_series.items()}
    except Exception as e:
      print(f"Erro ao obter dados para {symbol}: {e}")

  return data

# Simulated Annealing
def simulated_annealing(data, initial_solution, initial_temp, cooling_rate, num_iterations):
  current_solution = initial_solution
  current_cost = evaluate_portfolio(current_solution, data)
  best_solution = current_solution
  best_cost = current_cost
  temp = initial_temp

  for i in range(num_iterations):
    new_solution = current_solution.copy()

    # Começar com um portfólio inicial e realizar pequenas modificações, reavaliando a solução a cada passo.
    new_cost = evaluate_portfolio(new_solution, data)
    delta_cost = new_cost[0] - current_cost[0]

    # Aceitar  portfólios  subótimos  com  uma  probabilidade  que  diminui  ao  longo  do  tempo,  permitindo  a  exploração  do espaço de soluções.
    if delta_cost < 0 or random.random() < math.exp(-delta_cost / temp):
      current_solution = new_solution
      current_cost = new_cost

    if new_cost[0] < best_cost[0]:
      best_solution = new_solution
      best_cost = new_cost

    temp = exponential_decay(temp, cooling_rate)

  return best_solution, best_cost

def main():
  # Símbolos dos ativos que você deseja incluir no portfólio
  assets = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA"]
  n_assets = len(assets)

  # Substitua 'YOUR_API_KEY' pela sua chave de API Alpha Vantage
  api_key = '4KR9RHTR37YQ1XY5'

  # Obtém os dados de preços de fechamento dos ativos
  asset_data = get_alpha_vantage(assets, api_key)

  # Cria um DataFrame a partir dos dados obtidos
  data = pd.DataFrame(asset_data)

  # Parâmetros do Simulated Annealing
  initial_solution = np.random.random(n_assets)
  initial_temp = 1000
  cooling_rate = 0.995
  num_iterations = 10000

  start_time = time.time()
  best_solution, best_cost = simulated_annealing(data, initial_solution, initial_temp, cooling_rate, num_iterations)
  end_time = time.time()

  print("Melhor alocação de ativos:")
  print(assets)
  print(best_solution)
  best_return, best_risk = evaluate_portfolio(best_solution, data)
  print(f"Retorno anual: {best_return * 100:.2f}%")
  print(f"Risco anual: {best_risk * 100:.2}f%")
  print(f"Tempo de execução: {end_time - start_time:.2f} segundos")

if __name__ == "__main__":
  main()


Melhor alocação de ativos:
['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
[0.81580206 0.35885258 0.76473359 0.90314    0.26028357]
Retorno anual: 3.81%
Risco anual: nanf%
Tempo de execução: 8.51 segundos


# Comparação dos Algoritmos GA vs. SA

Tempo estimado de execução, 5 minutos para num_executions = 10

In [6]:
import matplotlib.pyplot as plt

# Configure os parâmetros do experimento
num_executions = 10  # Número de execuções para cada algoritmo

# Definir Métricas de Desempenho
tempos_de_execucao_ga = []
lucros_obtidos_ga = []
tempos_de_execucao_sa = []
lucros_obtidos_sa = []

# Símbolos dos ativos que você deseja incluir no portfólio
assets = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA"]
n_assets = len(assets)

# Substitua 'YOUR_API_KEY' pela sua chave de API Alpha Vantage
api_key = '4KR9RHTR37YQ1XY5'

# Obtém os dados de preços de fechamento dos ativos
asset_data = get_alpha_vantage(assets, api_key)

# Cria um DataFrame a partir dos dados obtidos
data = pd.DataFrame(asset_data)

# Parâmetros do Simulated Annealing
initial_solution = np.random.random(n_assets)
initial_temp = 1000
cooling_rate = 0.995
num_iterations = 10000

# Exemplo de uso
for _ in range(num_executions):
    # GA
    tempo_inicio_ga = time.time()
    best_solution_ga = genetic_algorithm(n_assets, data)
    tempo_fim_ga = time.time()
    
    # Verifique se há dados disponíveis antes de calcular os índices de máximo
    if data.shape[0] > 0:
        best_return_ga, best_risk_ga = evaluate_portfolio(best_solution_ga, data)
        lucros_obtidos_ga.append(best_return_ga)
        tempos_de_execucao_ga.append(tempo_fim_ga - tempo_inicio_ga)

    # SA
    tempo_inicio_sa = time.time()
    best_solution_sa, best_cost_sa = simulated_annealing(data, initial_solution, initial_temp, cooling_rate, num_iterations)
    tempo_fim_sa = time.time()

    # Verifique se há dados disponíveis antes de calcular os índices de máximo
    if data.shape[0] > 0:
        best_return_sa, best_risk_sa = evaluate_portfolio(best_solution_sa, data)
        lucros_obtidos_sa.append(best_return_sa)
        tempos_de_execucao_sa.append(tempo_fim_sa - tempo_inicio_sa)


# Plotar Gráficos
plt.figure(figsize=(12, 6))

# Gráfico de barras para o tempo de execução
plt.subplot(1, 2, 1)
plt.plot(range(num_executions), tempos_de_execucao_ga, label='Algoritmo GA')
plt.plot(range(num_executions), tempos_de_execucao_sa, label='Algoritmo SA')
plt.xlabel('Número de Execuções')
plt.ylabel('Tempo de Execução (s)')
plt.title('Tempo de Execução vs. Número de Execuções')
plt.legend()

# Gráfico de barras para o lucro obtido
plt.subplot(1, 2, 2)
plt.plot(range(num_executions), lucros_obtidos_ga, label='Algoritmo GA')
plt.plot(range(num_executions), lucros_obtidos_sa, label='Algoritmo SA')
plt.xlabel('Número de Execuções')
plt.ylabel('Lucro Obtido')
plt.title('Lucro Obtido vs. Número de Execuções')
plt.legend()

plt.tight_layout()
plt.show()

ValueError: attempt to get argmax of an empty sequence